## Day 7 Proyect: Hangman

### Hangman 🧑‍🦯​

En este proyecto implementamos un **juego del ahorcado** (Hangman) en Python.  
El juego sigue estos pasos:
1. Solicitar una palabra aleatoria mediante una API.
2. Solicitar al usuario que ingrese una letra y convertirla a minúsculas.
3. Comparar la letra ingresada con cada letra de la palabra elegida y actualizar la visualización (mostrando las letras acertadas y guiones bajos para las no acertadas).
4. Llevar un conteo de vidas (inicialmente 6). Cada vez que el usuario falle, se resta una vida y se muestra el avance del dibujo del ahorcado (ASCII art).
5. Finalizar el juego cuando se acierte la palabra completa o se agoten las vidas, mostrando mensajes adecuados.


```
88                                                                            
88                                                                            
88                                                                            
88,dPPYba,  ,adPPYYba, 8b,dPPYba,   ,adPPYb,d8 88,dPYba,,adPYba,  ,adPPYYba, 8b,dPPYba,  
88P'    "8a ""     `Y8 88P'   `"8a a8"    `Y88 88P'   "88"    "8a ""     `Y8 88P'   `"8a 
88       88 ,adPPPPP88 88       88 8b       88 88      88      88 ,adPPPPP88 88       88 
88       88 88,    ,88 88       88 "8a,   ,d88 88      88      88 88,    ,88 88       88 
88       88 `"8bbdP"Y8 88       88  `"YbbdP"Y8 88      88      88 `"8bbdP"Y8 88       88 
                                    aa,    ,88                                
                                     "Y8bbdP"  
```                               

#### 1. Importación de Módulos y Función para Obtener la Palabra

Importamos los módulos necesarios (`requests` y `random`) y definimos la función `obtener_palabra()` para solicitar una palabra aleatoria desde la API.  

Para obtener la palabra a adivinar, en lugar de usar un módulo interno, haremos uso de la API gratuita [Random Word API](https://random-word-api.herokuapp.com/).  
Si la API falla, se usará una palabra por defecto.

In [1]:
import random
import requests

# Definir stages manualmente
stages = [
    '''
     _______
    |/      |
    |      ( )
    |      /|\\
    |      / \\
    |
____|___
    ''',
    """
     _______
    |/      |
    |      ( )
    |      /|\\
    |      / 
    |
____|___
    """,
    """
     _______
    |/      |
    |      ( )
    |      /|\\
    |      
    |
____|___
    """,
    """
     _______
    |/      |
    |      ( )
    |      /|
    |      
    |
____|___
    """,
    """
     _______
    |/      |
    |      ( )
    |       |
    |      
    |
____|___
    """,
    """
     _______
    |/      |
    |      ( )
    |      
    |      
    |
____|___
    """,
    """
     _______
    |/      |
    |      
    |      
    |      
    |
____|___
    """
]

In [2]:
def obtener_palabra():
    """
    Solicita una palabra aleatoria a la Random Word API.
    Se convierte la palabra a minúsculas para asegurar consistencia.
    En caso de error, retorna 'default'.
    """
    try:
        respuesta = requests.get("https://random-word-api.herokuapp.com/word?number=1")
        if respuesta.status_code == 200:
            palabra = respuesta.json()[0]
            return palabra.lower()  # Convertir a minúsculas para consistencia
        else:
            print("Error en la solicitud a la API. Usando palabra por defecto.")
            return "riveraderrick"
    except Exception as e:
        print("Excepción al obtener palabra:", e)
        return "riveraderrick"

# Obtener la palabra a adivinar
chosen_word = obtener_palabra()
print(f"(DEBUG) La palabra elegida es: {chosen_word} ({len(chosen_word)} letras)")

(DEBUG) La palabra elegida es: orthopteroids (13 letras)


#### 2. Configuración Inicial y Visualización

Creamos un **placeholder** que mostrará guiones bajos (_) para cada letra de la palabra elegida, permitiendo al usuario saber cuántas letras debe adivinar.


In [3]:
# Crear un placeholder: si la palabra es "apple", se mostrará "_____"
placeholder = "_" * len(chosen_word)
print(f"Palabra a adivinar: {placeholder} ({len(chosen_word)} letras)" )

Palabra a adivinar: _____________ (13 letras)


#### 3. Solicitar Letra y Actualizar la Visualización

- Pedimos al usuario que adivine una letra y convertimos su respuesta a minúsculas.
- Recorremos la palabra elegida para construir el string `display`, que mostrará la letra si se adivinó o un guión bajo si no.
- Mantenemos una lista `correct_letters` para recordar las letras adivinadas correctamente.


In [4]:
# Solicitar letra al usuario y convertirla a minúsculas
guess = input(f"Adivina una letra de las {len(chosen_word)}:").lower()

# Inicializar variables para seguimiento
display = ""
correct_letters = []

# Actualizar display basado en la letra adivinada
for letter in chosen_word:
    if letter == guess or letter in correct_letters:
        display += letter
        if letter == guess and letter not in correct_letters:
            correct_letters.append(letter)
    else:
        display += "_"
print(f"Estado actual: {display}")


Estado actual: o___o____o___


#### 4. Control de Vidas y Feedback Visual

- Se crea la variable `lives` (inicialmente 6).
- Si la letra adivinada no se encuentra en la palabra, se resta una vida y se muestra un mensaje junto con el dibujo del ahorcado (ASCII art).
  
*Nota:* El ASCII art se puede incluir manualmente o definir una lista `stages` en el mismo notebook.


In [5]:
# Inicializar vidas
lives = 6

# Si la letra adivinada no está en la palabra, restar una vida
if guess not in chosen_word:
    lives -= 1
    print(f"¡La letra '{guess}' no está en la palabra! Pierdes una vida.")
    print(stages[lives])
else:
    print("¡Bien hecho, esa letra está en la palabra!")
    print(stages[lives])
    
print(f"Te quedan {lives} de 6 vidas.")


¡Bien hecho, esa letra está en la palabra!

     _______
    |/      |
    |      
    |      
    |      
    |
____|___
    
Te quedan 6 de 6 vidas.


#### 5. Bucle Principal del Juego

Integramos la lógica del juego dentro de un bucle `while` que permite al usuario seguir adivinando hasta:
- Adivinar la palabra completa (no queden guiones bajos en `display`), o  
- Agotar las vidas (lives == 0).

Se incluye verificación para evitar letras repetidas y se actualiza el estado de la palabra en cada iteración.


In [6]:
# Variable de control del juego
game_over = False

# Función para actualizar el display en función de las letras adivinadas
def actualizar_display(chosen_word, correct_letters):
    resultado = ""
    for letter in chosen_word:
        if letter in correct_letters:
            resultado += letter
        else:
            resultado += "_"
    return resultado

# Bucle principal del juego
display = ""
correct_letters = []
lives = 6

while not game_over:
    guess = input(f"Adivina una letra de las {len(chosen_word)}:").lower()
    
    # Verificar si la letra ya fue adivinada
    if guess in correct_letters:
        print(f"Ya has adivinado la letra '{guess}'. Intenta con otra.")
        continue
    
    # Actualizar lista de letras correctas o restar vidas
    if guess in chosen_word:
        correct_letters.append(guess)
        print(f"¡Bien hecho! La letra '{guess}' está en la palabra.​✔️​")
        print(stages[lives])
    else:
        lives -= 1
        print(f"¡La letra '{guess}' no está en la palabra! Pierdes una vida.❌​")
        print(stages[lives])
    
    # Actualizar y mostrar el estado de la palabra
    display = actualizar_display(chosen_word, correct_letters)
    print("Estado actual:", display)
    print(f"Te quedan {lives} de 6 vidas.\n")
    
    # Condición de victoria
    if "_" not in display:
        print("¡Felicidades! Adivinaste la palabra. 🏆​")
        game_over = True
    # Condición de derrota
    elif lives == 0:
        print("¡Game Over! Se han agotado tus vidas.☠️")
        print(f"La palabra era: {chosen_word}")
        game_over = True


¡La letra 'x' no está en la palabra! Pierdes una vida.❌​

     _______
    |/      |
    |      ( )
    |      
    |      
    |
____|___
    
Estado actual: _____________
Te quedan 5 de 6 vidas.

¡La letra 'x' no está en la palabra! Pierdes una vida.❌​

     _______
    |/      |
    |      ( )
    |       |
    |      
    |
____|___
    
Estado actual: _____________
Te quedan 4 de 6 vidas.

¡La letra 'x' no está en la palabra! Pierdes una vida.❌​

     _______
    |/      |
    |      ( )
    |      /|
    |      
    |
____|___
    
Estado actual: _____________
Te quedan 3 de 6 vidas.

¡La letra 'x' no está en la palabra! Pierdes una vida.❌​

     _______
    |/      |
    |      ( )
    |      /|\
    |      
    |
____|___
    
Estado actual: _____________
Te quedan 2 de 6 vidas.

¡La letra 'x' no está en la palabra! Pierdes una vida.❌​

     _______
    |/      |
    |      ( )
    |      /|\
    |      / 
    |
____|___
    
Estado actual: _____________
Te quedan 1 de 6 vidas.

In [7]:
# Variable de control del juego
game_over = False

# Función para actualizar el display en función de las letras adivinadas
def actualizar_display(chosen_word, correct_letters):
    resultado = ""
    for letter in chosen_word:
        if letter in correct_letters:
            resultado += letter
        else:
            resultado += "_"
    return resultado

# Bucle principal del juego
display = ""
correct_letters = []
lives = 6

while not game_over:
    guess = input("Adivina una letra: ").lower()
    
    # Verificar si la letra ya fue adivinada
    if guess in correct_letters:
        print(f"Ya has adivinado la letra '{guess}'. Intenta con otra.")
        continue
    
    # Actualizar lista de letras correctas o restar vidas
    if guess in chosen_word:
        correct_letters.append(guess)
        print(f"¡Bien hecho! La letra '{guess}' está en la palabra.​✔️​")
        print(stages[lives])
    else:
        lives -= 1
        print(f"¡La letra '{guess}' no está en la palabra! Pierdes una vida.❌​")
        print(stages[lives])
    
    # Actualizar y mostrar el estado de la palabra
    display = actualizar_display(chosen_word, correct_letters)
    print("Estado actual:", display)
    print(f"Te quedan {lives} de 6 vidas.")
    
    # Condición de victoria
    if "_" not in display:
        print("¡Felicidades! Adivinaste la palabra. 🏆​")
        game_over = True
    # Condición de derrota
    elif lives == 0:
        print("¡Game Over! Se han agotado tus vidas.☠️")
        print(f"La palabra era: {chosen_word}")
        game_over = True


¡Bien hecho! La letra 'o' está en la palabra.​✔️​

     _______
    |/      |
    |      
    |      
    |      
    |
____|___
    
Estado actual: o___o____o___
Te quedan 6 de 6 vidas.
¡Bien hecho! La letra 'r' está en la palabra.​✔️​

     _______
    |/      |
    |      
    |      
    |      
    |
____|___
    
Estado actual: or__o___ro___
Te quedan 6 de 6 vidas.
¡Bien hecho! La letra 't' está en la palabra.​✔️​

     _______
    |/      |
    |      
    |      
    |      
    |
____|___
    
Estado actual: ort_o_t_ro___
Te quedan 6 de 6 vidas.
¡Bien hecho! La letra 'h' está en la palabra.​✔️​

     _______
    |/      |
    |      
    |      
    |      
    |
____|___
    
Estado actual: ortho_t_ro___
Te quedan 6 de 6 vidas.
Ya has adivinado la letra 'o'. Intenta con otra.
¡Bien hecho! La letra 'p' está en la palabra.​✔️​

     _______
    |/      |
    |      
    |      
    |      
    |
____|___
    
Estado actual: orthopt_ro___
Te quedan 6 de 6 vidas.
Ya has adivinado

#### Conclusión

En este proyecto hemos aplicado conceptos fundamentales de Python, tales como:
- **Bucles y Condicionales:**  
  Uso de `while` y estructuras `if-elif-else` para controlar el flujo del juego.
- **Funciones:**  
  Creación de la función `actualizar_display()` para modularizar la actualización de la visualización.
- **Manejo de Listas y Cadenas:**  
  Uso de listas para almacenar las letras adivinadas y cadenas para construir el estado actual de la palabra.
- **Integración de APIs:**  
  Uso de la librería `requests` para obtener una palabra aleatoria desde una API, manteniendo la fuente oculta al usuario final.
- **Feedback al Usuario:**  
  Proporcionar mensajes claros y visuales (incluyendo ASCII art) que mejoran la experiencia de juego.

Este enfoque no solo refuerza los fundamentos de Python, sino que también demuestra habilidades en la integración de recursos externos y en la construcción de aplicaciones interactivas.
